# Kaggle Submission

In [1]:
# import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import json
%matplotlib inline

In [2]:
coeff=pd.read_csv("../datasets/model_coefficients.csv")

In [3]:
coeff

,Feature,Coefficient Value,Category
0,hdb_age,-3437.181384,hdb_age
1,mid,3358.043011,storey
2,max_floor_lvl,2798.384944,storey
3,floor_area_sqft,308.446031,floor_area_sqft
4,pri_sch_nearest_distance,10.590783,sch
...,...,...,...
79,full_flat_type_EXECUTIVE Apartment,99554.687502,full_flat_type
80,full_flat_type_EXECUTIVE Maisonette,130652.626938,full_flat_type
81,full_flat_type_EXECUTIVE Premium Apartment,74852.800236,full_flat_type
82,full_flat_type_EXECUTIVE Premium Maisonette,162809.690490,full_flat_type


In [5]:
kaggle_test=pd.read_csv("../datasets/test.csv")
kaggle_test

C:\Users\songk\AppData\Local\Temp\ipykernel_26100\1072759571.py:1: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  kaggle_test=pd.read_csv("../datasets/test.csv")


,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
0,114982,2012-11,YISHUN,4 ROOM,173,YISHUN AVE 7,07 TO 09,84.0,Simplified,1987,...,92,0,1.433681,103.832924,156.322353,Ahmad Ibrahim Secondary School,218,0,1.436235,103.829987
1,95653,2019-08,JURONG WEST,5 ROOM,986C,JURONG WEST ST 93,04 TO 06,112.0,Premium Apartment,2008,...,45,0,1.339244,103.698896,739.371688,Jurong West Secondary School,199,0,1.335256,103.702098
2,40303,2013-10,ANG MO KIO,3 ROOM,534,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,...,36,0,1.371893,103.851811,305.071191,Anderson Secondary School,245,0,1.374242,103.851430
3,109506,2017-10,WOODLANDS,4 ROOM,29,MARSILING DR,01 TO 03,97.0,New Generation,1979,...,54,0,1.434423,103.773698,433.454591,Woodlands Secondary School,188,0,1.439183,103.774499
4,100149,2016-08,BUKIT BATOK,4 ROOM,170,BT BATOK WEST AVE 8,16 TO 18,103.0,Model A,1985,...,40,0,1.349195,103.741000,217.295361,Bukit Batok Secondary School,223,0,1.348351,103.740873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16732,23347,2016-06,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,04 TO 06,82.0,New Generation,1980,...,63,0,1.376399,103.843166,320.543569,Yio Chu Kang Secondary School,188,0,1.378263,103.841809
16733,54003,2017-08,PUNGGOL,4 ROOM,305A,PUNGGOL RD,16 TO 18,92.0,Premium Apartment,2012,...,60,0,1.405333,103.905387,423.996711,Punggol Secondary School,194,0,1.402126,103.909119
16734,128921,2018-03,CHOA CHU KANG,5 ROOM,686D,CHOA CHU KANG CRES,04 TO 06,110.0,Improved,2002,...,54,0,1.402823,103.746993,560.399339,Unity Secondary School,212,0,1.399642,103.746036
16735,69352,2020-11,BISHAN,4 ROOM,167,BISHAN ST 13,07 TO 09,84.0,Simplified,1988,...,45,1,1.349783,103.854529,324.050692,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892


In [6]:
kaggle_test.columns=kaggle_test.columns.str.strip().str.lower() 

In [7]:
kaggle_test[kaggle_test.columns[kaggle_test.isnull().any()]].isnull().sum()

mall_nearest_distance       84
mall_within_500m         10292
mall_within_1km           2786
mall_within_2km            213
hawker_within_500m       10755
hawker_within_1km         6729
hawker_within_2km         3254
dtype: int64

In [8]:
#check if there is any discrepancy in minimum distance larger than 500m/1km/2km for rows with missing hawker
print(f"Minimium distance greater than 500m: \
{round(kaggle_test[kaggle_test['hawker_within_500m'].isnull()]['hawker_nearest_distance'].min(),3)}m")

print(f"Minimium distance greater than 1km: \
{round(kaggle_test[kaggle_test['hawker_within_1km'].isnull()]['hawker_nearest_distance'].min(),3)}m")

print(f"Minimium distance greater than 2km: \
{round(kaggle_test[kaggle_test['hawker_within_2km'].isnull()]['hawker_nearest_distance'].min(),3)}m")

Minimium distance greater than 500m: 500.209m
Minimium distance greater than 1km: 1000.074m
Minimium distance greater than 2km: 2000.872m


We observe that there are no rows with missing hawker nearest distance. For the rows with missing hawker value, we see that the minimum distance greater than 500m, 1km and 2km are 500.209m, 1000.074m and 2000.872m respectively. This implies that the nulls in each of these 3 columns can be set as 0, since the minimum distance available is larger than the specified distance.

In [9]:
# replace all missing hawker value to 0
kaggle_test.loc[kaggle_test['hawker_within_500m'].isnull(),'hawker_within_500m']=0
kaggle_test.loc[kaggle_test['hawker_within_1km'].isnull(),'hawker_within_1km']=0
kaggle_test.loc[kaggle_test['hawker_within_2km'].isnull(),'hawker_within_2km']=0

In [10]:
#retrieve all malls in Singapore
url='https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore'
response = requests.get(url)
print('Status Code: ',response.status_code)
html = response.content
soup = bs(html, 'lxml')
soup

malls = soup.find_all("div",{'class':'div-col'})
list_of_malls=[]
for i in range(len(malls)):
    list_of_malls+=[x.text for x in malls[i].find_all("li")]
list_of_malls=[x[:x.find("[")] if x.find("[")!=-1 else x for x in list_of_malls]
list_of_malls

Status Code:  200


['100 AM',
 '313@Somerset',
 'Aperia',
 'Balestier Hill Shopping Centre',
 'Bugis Cube',
 'Bugis Junction',
 'Bugis+',
 'Capitol Piazza',
 'Cathay Cineleisure Orchard',
 'Clarke Quay Central',
 'The Centrepoint',
 'City Square Mall',
 'City Gate Mall',
 'CityLink Mall',
 'Duo',
 'Far East Plaza',
 'Funan',
 'Great World City',
 'HDB Hub',
 'Holland Village Shopping Mall',
 'ION Orchard',
 'Junction 8',
 'Knightsbridge',
 'Liat Towers',
 'Lucky Plaza',
 'Marina Bay Sands',
 'The Shoppes at Marina Bay Sands',
 'Marina Bay Link Mall',
 'Marina Square',
 'Millenia Walk',
 'Mustafa Shopping Centre',
 'Ngee Ann City',
 'Orchard Central',
 'Orchard Gateway',
 'Orchard Plaza',
 'Midpoint Orchard',
 'Palais Renaissance',
 "People's Park Centre",
 "People's Park Complex",
 'Plaza Singapura',
 'PoMo',
 'Raffles City',
 'Scotts Square',
 'Shaw House and Centre',
 'Sim Lim Square',
 'Singapore Shopping Centre',
 'The South Beach',
 'Square 2',
 'Sunshine Plaza',
 'Suntec City',
 'Tanglin Mall',
 'T

In [11]:
# Obtaining Mall Coordinates in Singapore
list_of_malls
mall_name = []
mall_lat = []
mall_long = []

for i in range(0, len(list_of_malls)):
    query_address = list_of_malls[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)
    data_mall=json.loads(resp.content)
    
    if data_mall['found'] != 0:
        mall_name.append(query_address)
        mall_lat.append(float(data_mall["results"][0]["LATITUDE"]))
        mall_long.append(float(data_mall["results"][0]["LONGITUDE"]))

        print (str(query_address)+" ,Lat: "+data_mall['results'][0]['LATITUDE'] +" Long: "+data_mall['results'][0]['LONGITUDE'])

    else:
        print (f"No Results for {list_of_malls[i]}")

100 AM ,Lat: 1.27468281482263 Long: 103.843488359469
313@Somerset ,Lat: 1.30100656917241 Long: 103.838246592796
Aperia ,Lat: 1.3097112065077 Long: 103.864326436447
Balestier Hill Shopping Centre ,Lat: 1.32559594839311 Long: 103.842571612968
Bugis Cube ,Lat: 1.2981408343975 Long: 103.855635339249
Bugis Junction ,Lat: 1.2991371723215 Long: 103.855450325604
Bugis+ ,Lat: 1.30095171530648 Long: 103.855172625542
Capitol Piazza ,Lat: 1.29307884763132 Long: 103.851261982149
Cathay Cineleisure Orchard ,Lat: 1.30152101873533 Long: 103.836429655016
No Results for Clarke Quay Central
The Centrepoint ,Lat: 1.30145045537088 Long: 103.840034074858
City Square Mall ,Lat: 1.3113827185663 Long: 103.856626855768
No Results for City Gate Mall
CityLink Mall ,Lat: 1.29260349413333 Long: 103.854316064564
Duo ,Lat: 1.29953434891664 Long: 103.85840168774
Far East Plaza ,Lat: 1.30722100162543 Long: 103.833805646681
Funan ,Lat: 1.29134759697794 Long: 103.849989790085
Great World City ,Lat: 1.29342272650003 Long:

We observe 9 malls that were unable to retrieve latitude and longitude values, this missing information will be added manually by looking up online as most of the information has been compiled from OneMap API.

In [12]:
#add in latitude and longitude manually as search criteria not exactly matching between Wikipedia and OneMAP API
missing_malls={
    'Mall': ['Clarke Quay Central','City Gate Mall','Holland Village Shopping Mall','Mustafa Shopping Centre','PoMo','Shaw House and Centre','KINEX','Paya Lebar Quarter (PLQ)','OD Mall'],
    'Latitude': [1.2882413,1.30231590504573,1.31027747574118,1.31011213190394,1.300058,1.305512,1.31479,1.317369,1.3380],
    'Longitude': [103.846430401652,103.862331661034,103.795371163103,103.855290873926,103.849079,103.831755,103.89464,103.893266,103.7934]
}

mall_name += missing_malls['Mall']
mall_lat += missing_malls['Latitude']
mall_long += missing_malls['Longitude']

In [13]:
#https://www.geeksforgeeks.org/program-distance-two-points-earth/

#calculate distance between 2 points using Haversine formula

from math import radians, cos, sin, asin, sqrt
def dist(lat1, lon1, lat2, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c *r*1000)

In [14]:
missing_lat=[x[41] for x in kaggle_test[kaggle_test['mall_nearest_distance'].isnull()].values]
missing_long=[x[42] for x in kaggle_test[kaggle_test['mall_nearest_distance'].isnull()].values]

In [15]:
key=kaggle_test[kaggle_test['mall_nearest_distance'].isnull()]['mall_nearest_distance'].index

In [16]:
key

Int64Index([  441,   481,   579,   661,   782,  1127,  1278,  1632,  1705,
             1968,  2178,  2288,  2869,  2981,  3080,  3548,  3780,  4130,
             4199,  4413,  4558,  4999,  5141,  5333,  5643,  5689,  5901,
             6152,  6158,  6212,  6307,  6877,  7059,  7210,  7465,  7501,
             7817,  7909,  8005,  8310,  8416,  8591,  8915,  9188,  9198,
             9373,  9480,  9633,  9651, 10095, 10227, 10525, 11847, 11848,
            11856, 12005, 12290, 12299, 12461, 12512, 12816, 12892, 12908,
            12958, 12990, 13202, 13472, 13559, 13818, 13865, 13893, 13936,
            13975, 14154, 14328, 14645, 14803, 15180, 15202, 15276, 15745,
            16294, 16609, 16695],
           dtype='int64')

In [17]:
def mall_nearest_dist(latitude,longitude):
    distance=[]
    for i in range(len(latitude)):
        x=[]
        for j in range(len(mall_name)):
            x.append(dist(latitude[i],longitude[i],mall_lat[j],mall_long[j]))
        distance.append(pd.DataFrame(x).min().values[0])
    return distance

distance=pd.DataFrame(mall_nearest_dist(missing_lat,missing_long)).rename(columns={0:'mall_nearest_distance'})
distance.rename(index={x:key[x] for x in range(len(distance))},inplace=True)

def mall_dist(x):
    if pd.isna(kaggle_test['mall_nearest_distance'][x]):
        return distance['mall_nearest_distance'][x]
    else:
        pass

for i in range(len(kaggle_test)):
    if i in key.values:
        kaggle_test.loc[i,'mall_nearest_distance']=mall_dist(i)

In [18]:
def mall_500m(latitude,longitude):
    n_malls_500m=[]
    for i in range(len(latitude)):
        x=[]
        for j in range(len(mall_name)):
            #x.append(dist(missing.values[0][2],missing.values[0][3],mall_lat[i],mall_long[i]))
            x.append(dist(latitude[i],longitude[i],mall_lat[j],mall_long[j]))
        x=pd.DataFrame(x).rename(columns={0:'dist'})
        n_malls_500m.append(x[x['dist']<=500].count().values[0])
    return n_malls_500m

n_malls_500m=pd.DataFrame(mall_500m(missing_lat,missing_long)).rename(columns={0:'mall_within_500m'})
n_malls_500m.rename(index={x:key[x] for x in range(len(distance))},inplace=True)


for i in range(len(kaggle_test)):
    if i in key.values:
        kaggle_test.loc[i,'mall_within_500m']=n_malls_500m['mall_within_500m'][i]

In [19]:
def mall_1km(latitude,longitude):
    n_malls_1km=[]
    for i in range(len(latitude)):
        x=[]
        for j in range(len(mall_name)):
            #x.append(dist(missing.values[0][2],missing.values[0][3],mall_lat[i],mall_long[i]))
            x.append(dist(latitude[i],longitude[i],mall_lat[j],mall_long[j]))
        x=pd.DataFrame(x).rename(columns={0:'dist'})
        n_malls_1km.append(x[x['dist']<=1000].count().values[0])
    return n_malls_1km

n_malls_1km=pd.DataFrame(mall_1km(missing_lat,missing_long)).rename(columns={0:'mall_within_1km'})
n_malls_1km.rename(index={x:key[x] for x in range(len(distance))},inplace=True)


for i in range(len(kaggle_test)):
    if i in key.values:
        kaggle_test.loc[i,'mall_within_1km']=n_malls_1km['mall_within_1km'][i]

In [20]:
def mall_2km(latitude,longitude):
    n_malls_2km=[]
    for i in range(len(latitude)):
        x=[]
        for j in range(len(mall_name)):
            #x.append(dist(missing.values[0][2],missing.values[0][3],mall_lat[i],mall_long[i]))
            x.append(dist(latitude[i],longitude[i],mall_lat[j],mall_long[j]))
        x=pd.DataFrame(x).rename(columns={0:'dist'})
        n_malls_2km.append(x[x['dist']<=2000].count().values[0])
    return n_malls_2km

n_malls_2km=pd.DataFrame(mall_2km(missing_lat,missing_long)).rename(columns={0:'mall_within_2km'})
n_malls_2km.rename(index={x:key[x] for x in range(len(distance))},inplace=True)


for i in range(len(kaggle_test)):
    if i in key.values:
        kaggle_test.loc[i,'mall_within_2km']=n_malls_2km['mall_within_2km'][i]

In [21]:
#check if there is any discrepancy in minimum mall distance larger than 500m/1km/2km for rows with missing mall count
print(f"Minimium mall distance greater than 500m: \
{round(kaggle_test[kaggle_test['mall_within_500m'].isnull()]['mall_nearest_distance'].min(),3)}m")

print(f"Minimium mall distance greater than 1km: \
{round(kaggle_test[kaggle_test['mall_within_1km'].isnull()]['mall_nearest_distance'].min(),3)}m")

print(f"Minimium mall distance greater than 2km: \
{round(kaggle_test[kaggle_test['mall_within_2km'].isnull()]['mall_nearest_distance'].min(),3)}m")

Minimium mall distance greater than 500m: 500.075m
Minimium mall distance greater than 1km: 1000.023m
Minimium mall distance greater than 2km: 2000.176m


We observe that there are no rows with missing mall nearest distance. For the rows with missing mall value, we see that the minimum distance greater than 500m, 1km and 2km are 500.075m, 1000.023m and 2000.176m respectively. This implies that the nulls in each of these 3 columns can be set as 0, since the minimum distance available is larger than the specified distance.

In [22]:
kaggle_test.loc[kaggle_test['mall_within_500m'].isnull(),'mall_within_500m']=0
kaggle_test.loc[kaggle_test['mall_within_1km'].isnull(),'mall_within_1km']=0
kaggle_test.loc[kaggle_test['mall_within_2km'].isnull(),'mall_within_2km']=0

In [23]:
kaggle_test.isnull().sum().sum()

0

In [24]:
X = kaggle_test[['hdb_age','mid','max_floor_lvl','floor_area_sqft','pri_sch_nearest_distance','sec_sch_nearest_dist','hawker_nearest_distance','mrt_nearest_distance',
                'bus_interchange','mrt_interchange','mall_nearest_distance','planning_area','full_flat_type']]
X = pd.get_dummies(X, columns=['planning_area','full_flat_type'],drop_first=True)

In [25]:
# to filter out the extra columns in the train dataset, the train dataset has 3 additional columns not found in the test dataset

ex=list(set(coeff.Feature.values).difference(set(X.columns)))

coeff[~coeff['Feature'].isin(ex)]

,Feature,Coefficient Value,Category
0,hdb_age,-3437.181384,hdb_age
1,mid,3358.043011,storey
2,max_floor_lvl,2798.384944,storey
3,floor_area_sqft,308.446031,floor_area_sqft
4,pri_sch_nearest_distance,10.590783,sch
...,...,...,...
79,full_flat_type_EXECUTIVE Apartment,99554.687502,full_flat_type
80,full_flat_type_EXECUTIVE Maisonette,130652.626938,full_flat_type
81,full_flat_type_EXECUTIVE Premium Apartment,74852.800236,full_flat_type
82,full_flat_type_EXECUTIVE Premium Maisonette,162809.690490,full_flat_type


In [26]:
# additional columns in the train dataset
ex

['full_flat_type_2 ROOM DBSS',
 'full_flat_type_2 ROOM 2-room',
 'full_flat_type_4 ROOM Terrace']

In [27]:
#linear regression model coefficient
lr_coeff=206971.449092748

In [28]:
predicted_price=np.matmul(X, coeff[~coeff['Feature'].isin(ex)][['Coefficient Value']])+lr_coeff

C:\Users\songk\AppData\Local\Temp\ipykernel_26100\1110527849.py:1: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  predicted_price=np.matmul(X, coeff[~coeff['Feature'].isin(ex)][['Coefficient Value']])+lr_coeff


In [29]:
kaggle_submission=pd.concat([kaggle_test['id'],predicted_price],axis=1).rename(columns={0:'Predicted'})

In [30]:
kaggle_submission['id']=kaggle_submission['id'].map(str)

In [31]:
kaggle_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         16737 non-null  object 
 1   Predicted  16737 non-null  float64
dtypes: float64(1), object(1)
memory usage: 261.6+ KB


In [32]:
kaggle_submission.describe()

,Predicted
count,1.673700e+04
mean,4.480686e+05
std,1.333949e+05
min,5.559364e+04
25%,3.551607e+05
50%,4.344332e+05
75%,5.182420e+05
max,1.308890e+06


In [33]:
kaggle_submission.isnull().sum().sum()

0

In [34]:
kaggle_submission.to_csv("../datasets/kaggle_submission.csv",index=False)